## Import Libraries

In [ ]:
import FileManager
import WordCleaner
import Indexer
%load_ext autoreload
%autoreload 2

## Load Files

In [ ]:
dataset = FileManager.csvToDict('testing.csv')
datasets = [dataset]

## Stemming

In [ ]:
dataset = datasets[-1]
stemmed_dataset = {}
for row in dataset:
    stemmed_words = WordCleaner.stem(dataset[row], 'porter')
    stemmed_dataset[row] = stemmed_words
datasets.append(stemmed_dataset)

## OR Lemmatization

In [ ]:
dataset = datasets[-1]
lemmad_dataset = {}
for row in dataset:
    stemmed_words = WordCleaner.lemmatize(dataset[row])
    lemmad_dataset[row] = stemmed_words
datasets.append(lemmad_dataset)

## Removing Stopwords

In [ ]:
dataset = datasets[-1]
filtered_dataset = {}
for key in dataset:
    filtered_dataset[key] = WordCleaner.removeStopWords(dataset[key])
datasets.append(filtered_dataset)

## Creating the inverted index

In [ ]:
inverted_index = Indexer.getInvertedIndex(datasets[-1])
inverted_index

## Calculating tf-idf for the document

### using Scikit Learn

In [ ]:
(tfidf_matrix, df) = Indexer.calculateTF_IDF(datasets[-1])

df

### Manually

In [ ]:
(tfidf_matrix, df) = Indexer.calculateManualTF_IDF(datasets[-1])

df

## Write To File

In [ ]:
file_writer, file = FileManager.openCSVWriter('stemmed.csv',['id','text'])
for key in datasets[-1]:
    file_writer.writerow({'id': key, 'text': datasets[-1][key]})
file.close()